In [ ]:
# 최종 summary 전달 코드

from openai import OpenAI
import time


# -----------------------------------------------------------
# OpenAI 클라이언트
# -----------------------------------------------------------
client = OpenAI(api_key="")
MODEL_NAME = "gpt-4o-mini"
TEMPERATURE = 0.2
MAX_TOKENS = 700


# -----------------------------------------------------------
# 뉴스 요약 프롬프트 (TEXT 버전)
# -----------------------------------------------------------
SUMMARY_PROMPT = """
당신은 뉴스 요약 전문가입니다.

아래 기사 제목과 본문 내용을 읽고:

1) 기사 핵심 내용을 4문장으로 요약해 주세요.
2) 기사 핵심 키워드 5개를 bullet 형식으로 출력해 주세요.

출력 형식은 아래처럼 '텍스트'로만 출력하세요. JSON 금지.

예시 출력:
요약:
- 문장1
- 문장2
- 문장3
- 문장4

키워드:
- 키워드1
- 키워드2
- 키워드3
- 키워드4
- 키워드5

제목: {title}

본문:
{body}
"""


# -----------------------------------------------------------
# summarize_article()
# → LLM으로 요약 텍스트(raw text) 생성
# -----------------------------------------------------------
def summarize_article(title: str, content: str) -> str:

    if len(content) > 8000:
        content = content[:8000]

    prompt = SUMMARY_PROMPT.format(title=title, body=content)

    for attempt in range(3):
        try:
            resp = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": prompt}],
                temperature=TEMPERATURE,
                max_tokens=MAX_TOKENS,
            )

            raw = resp.choices[0].message.content
            return raw

        except Exception as e:
            if attempt == 2:
                raise RuntimeError(f"요약 생성 실패: {e}")
            time.sleep(0.5)


# -----------------------------------------------------------
# parse_summary_output()
# → raw text 를 DB 저장용 summary / keywords 로 분리
# -----------------------------------------------------------
def parse_summary_output(raw_text: str):
    lines = [line.strip() for line in raw_text.split("\n") if line.strip()]

    summary_lines = []
    keyword_lines = []
    mode = None

    for line in lines:
        if line.startswith("요약"):
            mode = "summary"
            continue

        elif line.startswith("키워드"):
            mode = "keywords"
            continue

        if mode == "summary" and line.startswith("-"):
            summary_lines.append(line.lstrip("- ").strip())

        elif mode == "keywords" and line.startswith("-"):
            keyword_lines.append(line.lstrip("- ").strip())

    summary_str = " ".join(summary_lines)
    keywords_str = ", ".join(keyword_lines)

    return summary_str, keywords_str


In [37]:
if __name__ == "__main__":

    test_title = "딥시크, 차세대 모델 개발에 '블랙웰' 칩 사용...내년 2월 출시 목표"
    test_content = """
    중국 딥시크가 미국의 수출 통제 대상인 엔비디아의 차세대 '블랙웰(Blackwell)' 칩 수천 개를 밀반입해 차세대 모델 개발에 활용했다는 주장이 나왔다. 이를 통한 새 모델은 내년 2월 설 연휴 이전 출시 목표인 것으로 알려졌다. 디 인포메이션은 10일(현지시간) 복수의 소식통을 인용, 딥시크가 해외 데이터센터를 거점으로 블랙웰 칩이 탑재된 서버를 합법적으로 구매한 뒤 엔비디아와 서버 제조사의 현장 검수를 마친 상태에서 서버를 분해해 중국으로 반입됐다고 보도했다. 이 과정에서 중국 세관에는 허위 신고가 이뤄지며, 분해된 서버 부품들은 중국 내 데이터센터로 옮겨져 딥시크와 체결한 임대 계약에 따라 현장에서 재조립되는 방식으로 운용된다. 이런 방식은 이른바 '유령 데이터센터'로 불리는 밀수 구조로, 서류상으로는 최종 사용자 추적이 불가능해 미국이 법적 조치를 취하기 어려운 구조다. 미국 법무부는 최근 엔비디아 칩 밀수 혐의로 3명을 기소했지만, 전문가들은 ""사실 입증이 어렵고 단속 자체도 쉽지 않다""라고 지적했다. 엔비디아는 ""데이터센터 해체 후 밀반입이 실제로 발생했다는 구체적 증거는 확인되지 않았다""라고 밝혔다. 하지만, 관련 제보에 대해서는 추적하겠다고 밝혔다. 딥시크는 지난 1월 추론 모델 'R1'을 선보이며 글로벌 AI 업계를 뒤흔들었다. 특히, 훈련 비용이 기존 모델 대비 현저히 낮았다는 점이 큰 화제를 불러일으켰다. 그러나 이후 신형 모델 발표는 지연됐다. 차세대 모델은 '희소 어텐션(sparse attention)' 기법을 채택하면서 개발 난도가 크게 높아진 것으로 알려졌다. 희소 어텐션은 필요한 연산만 활성화해 추론 비용을 크게 줄이는 구조인데, 블랙웰 칩은 이런 희소 연산에 최적화된 칩으로 평가된다. 따라서 블랙웰 없이는 차세대 모델 개발이 어렵다는 것이다. 이처럼 블랙웰 칩 확보로 차세대 모델은 내년 2월 설 연휴 전 출시가 목표인 것으로 알려졌다. 이는 당초 알려졌던 지난 5월보다 무려 9개월이 밀린 것이다. 그동안 딥시크는 엔비디아 고급 칩 확보가 어려워 기존 V3 모델의 업그레이드 버전을 내놓는 수준에 그쳤다. 이 과정에서 중국 정부의 권유로 중국산 칩을 모델 훈련에 활용하기도 했다. 출시는 더 늦어질 수도 있다. 량원펑 딥시크 창립자는 ""일정보다 성능이 우선""이라는 원칙을 고수하며 공식 발표 시점을 확정하지 않은 것으로 알려졌다. 한편, 엔비디아는 GPU의 위치를 파악할 수 있는 '위치 검증(Location Verification)' 기술을 시험 중이라고 밝혔다. 이 소프트웨어는 칩의 연산 패턴과 서버 간 통신 지연을 분석, 어느 국가에서 사용 중인지 추정하는 기술이다. 블랙웰 칩에 우선 적용되며, 고객이 선택적으로 활성화할 수 있는 옵션으로 제공될 예정이다. 이를 두고 중국 정부는 이미 지난 7월 엔비디아에 백도어 장치가 존재하지 않는다는 것을 입증하라고 요구한 바 있다. 엔비디아는 이런 점을 의식한 듯 ""등록된 시스템을 원격으로 제어하거나 조치를 취할 수 있는 기능은 없다""라며 ""원격 측정 데이터는 읽기 전용""이라고 강조했다. 

    """

    print("\n===== RAW OUTPUT =====")
    raw = summarize_article(test_title, test_content)
    print(raw)

    print("\n===== PARSED OUTPUT =====")
    summary, keywords = parse_summary_output(raw)
    print("SUMMARY:", summary)
    print("KEYWORDS:", keywords)


===== RAW OUTPUT =====
요약:
- 중국 딥시크가 엔비디아의 블랙웰 칩을 밀반입해 차세대 AI 모델 개발에 활용하고 있으며, 내년 2월 출시를 목표로 하고 있다.
- 딥시크는 해외 데이터센터에서 블랙웰 칩이 탑재된 서버를 합법적으로 구매한 후, 이를 분해해 중국으로 반입하는 방식으로 밀수 구조를 운영하고 있다.
- 전문가들은 미국의 법적 조치가 어렵다고 지적하며, 엔비디아는 밀반입에 대한 구체적 증거는 확인되지 않았다고 밝혔다.
- 딥시크의 차세대 모델은 희소 어텐션 기법을 채택하여 개발 난도가 높아졌으며, 성능 우선 원칙에 따라 출시 일정이 지연될 가능성도 있다.

키워드:
- 딥시크
- 블랙웰 칩
- 밀반입
- 차세대 모델
- 희소 어텐션

===== PARSED OUTPUT =====
SUMMARY: 중국 딥시크가 엔비디아의 블랙웰 칩을 밀반입해 차세대 AI 모델 개발에 활용하고 있으며, 내년 2월 출시를 목표로 하고 있다. 딥시크는 해외 데이터센터에서 블랙웰 칩이 탑재된 서버를 합법적으로 구매한 후, 이를 분해해 중국으로 반입하는 방식으로 밀수 구조를 운영하고 있다. 전문가들은 미국의 법적 조치가 어렵다고 지적하며, 엔비디아는 밀반입에 대한 구체적 증거는 확인되지 않았다고 밝혔다. 딥시크의 차세대 모델은 희소 어텐션 기법을 채택하여 개발 난도가 높아졌으며, 성능 우선 원칙에 따라 출시 일정이 지연될 가능성도 있다.
KEYWORDS: 딥시크, 블랙웰 칩, 밀반입, 차세대 모델, 희소 어텐션
